In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import DistanceMetric
from scipy.spatial.distance import mahalanobis
import os
import sys
sys.path.append('../models')
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
##########################################
######### Configure Metric Setting ####### 
##########################################
batch_size = 64
weight_path = './weights/cifar10_resnet18_v2.pt'
model = torch.load(weight_path)
criterion = nn.CrossEntropyLoss()
member_path = '/media/data1/hyunjun/cifar-10/train/'
nonmember_path = '/media/data1/hyunjun/cifar-10/test/'
data = 'Cifar-10'
##########################################
##########################################

In [4]:
mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225) 
if data == 'STL-10':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)
elif data == 'Cifar-10':
    mean, std = (0.491, 0.482, 0.447), (0.247, 0.243, 0.262)
elif data == 'CelebA':
    mean, std = (0.485, 0.456, 0.406), (0.229, 0.224, 0.225)

trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
])

model.to(device)
model.eval()

nonmember_set = torchvision.datasets.ImageFolder(root=nonmember_path, transform=trans)
nonmember_loader = DataLoader(nonmember_set, batch_size=batch_size, shuffle=True, drop_last=False,)
member_set = torchvision.datasets.ImageFolder(root=member_path, transform=trans)
member_loader = DataLoader(member_set, batch_size=batch_size, shuffle=True, drop_last=False,)

classes = member_set.classes

In [5]:
latent_vectors = [[] for _ in range(len(classes))]

model.eval()
with torch.no_grad():
    for data in member_loader:
        images, labels = data[0].to(device), data[1].to(device)
        features, _ = model(images)

        deep_features = features.cpu().numpy().tolist()
        actual = labels.cpu().numpy().tolist()
        
        for i in range(len(actual)):
            latent_vectors[actual[i]].append(list(deep_features[i]))


In [6]:
print("#####################################")
print("### Calculating Cosine Similarity ###")
print("#####################################")

for i in range(len(latent_vectors)):
    cssm = cosine_similarity(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(cssm):.4f}, min: {np.min(cssm):.4f}, mean: {np.mean(cssm):.4f}, var: {np.var(cssm):.4f}")

#####################################
### Calculating Cosine Similarity ###
#####################################
class: airplane, max: 1.0000, min: 0.4076, mean: 0.9018, var: 0.0034
class: automobile, max: 1.0000, min: 0.3453, mean: 0.8772, var: 0.0046
class: bird, max: 1.0000, min: 0.2345, mean: 0.8743, var: 0.0047
class: cat, max: 1.0000, min: 0.4297, mean: 0.9114, var: 0.0025
class: deer, max: 1.0000, min: 0.3930, mean: 0.9047, var: 0.0032
class: dog, max: 1.0000, min: 0.4179, mean: 0.8814, var: 0.0039
class: frog, max: 1.0000, min: 0.3080, mean: 0.8941, var: 0.0042
class: horse, max: 1.0000, min: 0.5353, mean: 0.9357, var: 0.0015
class: ship, max: 1.0000, min: 0.3156, mean: 0.8931, var: 0.0038
class: truck, max: 1.0000, min: 0.4879, mean: 0.9210, var: 0.0021


In [7]:
print("#####################################")
print("## Calculating Euclidean Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('euclidean')
for i in range(len(latent_vectors)):
    euclidean = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(euclidean):.4f}, min: {np.min(euclidean):.4f}, mean: {np.mean(euclidean):.4f}, var: {np.var(euclidean):.4f}")

#####################################
## Calculating Euclidean Distance ###
#####################################
class: airplane, max: 38.3830, min: 0.0000, mean: 11.4491, var: 12.1017
class: automobile, max: 34.4974, min: 0.0000, mean: 12.8696, var: 13.2395
class: bird, max: 35.7884, min: 0.0000, mean: 11.3693, var: 10.1061
class: cat, max: 41.1791, min: 0.0000, mean: 12.3374, var: 13.7583
class: deer, max: 40.5108, min: 0.0000, mean: 12.1666, var: 14.1131
class: dog, max: 40.6471, min: 0.0000, mean: 12.4478, var: 12.6667
class: frog, max: 35.5434, min: 0.0000, mean: 11.8385, var: 12.1216
class: horse, max: 42.2903, min: 0.0000, mean: 12.3608, var: 16.6867
class: ship, max: 32.9070, min: 0.0000, mean: 11.3099, var: 10.7442
class: truck, max: 39.1107, min: 0.0000, mean: 12.2118, var: 13.3494


In [8]:
print("#####################################")
print("## Calculating Manhattan Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('manhattan')
for i in range(len(latent_vectors)):
    manhattan = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(manhattan):.4f}, min: {np.min(manhattan):.4f}, mean: {np.mean(manhattan):.4f}, var: {np.var(manhattan):.4f}")

#####################################
## Calculating Manhattan Distance ###
#####################################
class: airplane, max: 106.3786, min: 0.0000, mean: 29.9948, var: 92.9097
class: automobile, max: 91.7327, min: 0.0000, mean: 33.1654, var: 91.3227
class: bird, max: 98.1471, min: 0.0000, mean: 29.5138, var: 74.8660
class: cat, max: 122.9356, min: 0.0000, mean: 32.3551, var: 109.5868
class: deer, max: 114.0511, min: 0.0000, mean: 31.7886, var: 108.0165
class: dog, max: 111.0948, min: 0.0000, mean: 32.2794, var: 94.3828
class: frog, max: 92.3577, min: 0.0000, mean: 30.6638, var: 86.5950
class: horse, max: 115.0586, min: 0.0000, mean: 32.3368, var: 123.5827
class: ship, max: 90.4588, min: 0.0000, mean: 29.3065, var: 77.0218
class: truck, max: 103.7976, min: 0.0000, mean: 31.7498, var: 96.1733


In [9]:
print("#####################################")
print("## Calculating Chebyshev Distance ###")
print("#####################################")
dist = DistanceMetric.get_metric('chebyshev')
for i in range(len(latent_vectors)):
    chebyshev = dist.pairwise(np.array(latent_vectors[i]))
    print(f"class: {classes[i]}, max: {np.max(chebyshev):.4f}, min: {np.min(chebyshev):.4f}, mean: {np.mean(chebyshev):.4f}, var: {np.var(chebyshev):.4f}")

#####################################
## Calculating Chebyshev Distance ###
#####################################
class: airplane, max: 23.4877, min: 0.0000, mean: 6.8023, var: 4.7904
class: automobile, max: 25.7819, min: 0.0000, mean: 7.8932, var: 6.7028
class: bird, max: 23.3473, min: 0.0000, mean: 6.8903, var: 4.6505
class: cat, max: 24.3177, min: 0.0000, mean: 7.3385, var: 5.2525
class: deer, max: 24.8609, min: 0.0000, mean: 7.2733, var: 5.7286
class: dog, max: 25.2622, min: 0.0000, mean: 7.5580, var: 5.7616
class: frog, max: 24.9020, min: 0.0000, mean: 7.1729, var: 5.5280
class: horse, max: 27.3233, min: 0.0000, mean: 7.3904, var: 6.9309
class: ship, max: 23.7469, min: 0.0000, mean: 6.8846, var: 5.1845
class: truck, max: 29.1943, min: 0.0000, mean: 7.3817, var: 6.1806
